In [127]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
import tensorflow as tf

from sklearn.model_selection import train_test_split
from skimage import color, exposure, filters, io, morphology, util
from math import sqrt
from numpy import loadtxt
from tensorflow import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Dropout, Flatten, MaxPooling2D, Reshape
from keras.optimizers import SGD

### TRANSFER LEARNING

1. Bangun sebuah model $\text{Convolutional Neural Network}$ dengan arsitektur dasar sebagai berikut :
    - Convolutional Layer ($25$ maps, kernel $3 \times 3$)
    - Pooling Layer ($2 \times 2$)
    - Activation Function ($\text{ReLU function}$)
    - Convolutional Layer ($50$ maps, kernel $3 \times 3$)
    - Pooling Layer ($2 \times 2$)
    - Activation Function ($\text{ReLU function}$)
    - Convolutional Layer ($100$ maps, kernel $3 \times 3$)
    - Pooling Layer ($2 \times 2$)
    - Activation Function ($\text{ReLU function}$)
    - Hidden Layer ($100$ neuron)
    - Activation Function ($\text{ReLU function}$)
    - Output Layer ($10$ kelas)
    - Activation Function ($\text{Softmax function}$)
    - Classification Result

  Dengan tambahan beberapa seperti:
  - Adam optimization
  - Early Stop

2. Implementasi model $\text{Convolutional Neural Network}$ tersebut pada dataset $\text{CIFAR 10}$ (ambil random hanya $1000$ citra per kelas) dengan inisial $\text{epoch} = 100$, $\text{batch number} = 10$, $\text{learning rate} = 0.1$, serta ratio data train dan data test sebanyak $60\% : 40\%$.

#### Mendefinisikan Fungsi Bantuan
Untuk memudahkan implementasi, berikut beberapa fungsi bantuan yang akan dipakai

##### Menghitung $\text{recall}$ atau $\text{sensitivity}$
$$\text{recall} = \frac{t_p}{t_p+t_n}$$

In [2]:
def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    return true_positives / (possible_positives + K.epsilon())

##### Menghitung $\text{precision}$
$$\text{precision} = \frac{t_p}{t_p + f_p}$$

In [3]:
def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    return true_positives / (predicted_positives + K.epsilon())

##### Menghitung $\text{specificity}$
$$\text{specificity} = \frac{t_n}{t_n + f_p}$$

In [4]:
def specificity(y_true, y_pred):
    true_negatives = K.sum(K.round(K.clip((1 - y_true) * (1 - y_pred), 0, 1)))
    possible_negatives = K.sum(K.round(K.clip(1 - y_true, 0, 1)))
    return true_negatives / (possible_negatives + K.epsilon())

##### Menghitung $f_{1}\text{ Score}$
Kita tau bahwa
$$f_{\beta} = (1 + \beta^{2}) \cdot \frac{\text{precision} \cdot \text{recall}}{(\beta^{2} \cdot \text{precision}) + \text{recall}}$$
Sehingga 
$$f_{1} = (1 + 1^{2}) \cdot \frac{\text{precision} \cdot \text{recall}}{(1^{2} \cdot \text{precision}) + \text{recall}}$$
$$f_{1} = (2) \cdot \frac{\text{precision} \cdot \text{recall}}{(\text{precision}) + \text{recall}}$$



In [5]:
def f1_score(y_true, y_pred):
    precision_val = precision(y_true, y_pred)
    recall_val = recall(y_true, y_pred)
    return 2*((precision_val * recall_val)/(precision_val + recall_val + K.epsilon()))

**n.b.: Semua fungsi pada bagian penyebut ditambah epsilon untuk menghindari dibagi $0$ dikarenakan memasukkan input sebanyak $0$**

##### Fungsi Bantuan untuk Melakukan Prediksi dan Menampilkan Metric

In [6]:
def predict_and_get_metrics(model, x_test, y_test, label):
    _, accuracy, sensitivity, specificity, score_f1 = model.evaluate(x_test, y_test, verbose=0)
    return pd.DataFrame([[accuracy, sensitivity, specificity, score_f1]], columns=['Accuracy', 'Sensitivity', 'Specificity', 'F1 Score'], index=[label])

def compare_train_and_test(model, x_train, y_train, x_test, y_test, extra_label=''):
    metric_train = predict_and_get_metrics(model, x_train, y_train, 'Data Train %s' % (extra_label))
    metric_test = predict_and_get_metrics(model, x_test, y_test, 'Data Test %s' % (extra_label))
    return pd.concat([metric_train, metric_test])

##### Fungsi Bantuan untuk _Load Dataset_

In [47]:
def load_dataset_cifar10():
	(trainX, trainY), (testX, testY) = keras.datasets.cifar10.load_data()
	return np.concatenate((trainX, testX), axis=0), np.concatenate((trainY, testY), axis=0)

#### Load Dataset

In [87]:
x, y = load_dataset_cifar10()
x.shape

(60000, 32, 32, 3)

#### Mengambil Dataset Sebanyak 1000 Per Kelasnya

In [122]:
idx_sample = np.array([],dtype=int)

for i in range(10):
    idx = (y == i).reshape(x.shape[0])
    idx_data = np.where(idx == True)
    sampled_list = np.random.choice(idx_data[0], size=1000, replace=False)
    idx_sample = np.concatenate((idx_sample, sampled_list), axis=0)

#### Melakukan Validasi Apakah Shape-nya Sudah Valid atau Belum

In [124]:
x_sample = x[idx_sample]
y_sample = y[idx_sample]
x_sample.shape, y_sample.shape

((10000, 32, 32, 3), (10000, 1))

#### Merubah Menjadi Matrix Binary untuk $y$

In [125]:
y_sample = tf.keras.utils.to_categorical(y_sample)

#### Memecah Menjadi 60 : 40 ~ Train : Test

In [128]:
x_train, x_test, y_train, y_test = train_test_split(x_sample, y_sample, test_size=0.4)

#### Membangun Model Convolutional Neural Network

In [129]:
convolutional_layers = []
## Convolutional Layers 25 map, 3 x 3, activation = relu
convolutional_layers.append(Conv2D(25, kernel_size=(3, 3), activation=tf.nn.relu, padding='SAME', kernel_initializer=tf.keras.initializers.HeUniform()))

## Pooling Layer 2 x 2
convolutional_layers.append(MaxPooling2D(2,2))

## Convolutional Layers 50 map, 3 x 3, activation = relu
convolutional_layers.append(Conv2D(50, kernel_size=(3, 3), activation=tf.nn.relu, padding='SAME', kernel_initializer=tf.keras.initializers.HeUniform()))

## Pooling Layer 2 x 2
convolutional_layers.append(MaxPooling2D(2,2))

## Convolutional Layers 100 map, 3 x 3, activation = relu
convolutional_layers.append(Conv2D(100, kernel_size=(3, 3), activation=tf.nn.relu, padding='SAME', kernel_initializer=tf.keras.initializers.HeUniform()))

## Pooling Layer 2 x 2
convolutional_layers.append(MaxPooling2D(2,2))

convolutional_layers.append(Flatten())

## Hidden Layer 100 node
convolutional_layers.append(Dense(100, activation=tf.nn.relu))

## Output 10 class
convolutional_layers.append(Dense(10, activation=tf.nn.softmax))

## Menginisialisasi instance model
convolutional_model = Sequential(convolutional_layers)

## Membangun model
convolutional_model.build(input_shape=x_train.shape)
convolutional_model.output_shape

(6000, 10)

#### Melakukan Kompilasi ke dalam Model

In [130]:
convolutional_model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss='categorical_crossentropy',
    metrics=[
      'accuracy',
      recall,
      specificity,
      f1_score
    ])